In [4]:
from mtpbox import *
import mtpbox.comm_modules as cm

from struct import Struct
from recordclass import recordclass
import re

#initialization of logging
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.DEBUG,datefmt="%H:%M:%S")




class Telegram(Struct):
    
    #packing flags for "telegram_format":
    #<little endial
    #>big endian
    #x - pad byte
    #c - char
    #h - signed int 2 bytes
    #H - unigned int 2 bytes
    #i - signed int 4 bytes
    #I - unigned int 4 bytes
    #i - signed 4 bytes integer
    #I - unigned 4 bytes integer
    #l - signed 8 bytes integer
    #L - unigned 8 bytes integer
    #f - floating point 4 bytes
    #d - floating point 8 bytes
    #Ns - string, N chars
    #P -  void *
    
    def __init__(self,telegram_format,telegram_name,telegram_data_names=None,telegram_data=None,comm_module=None,string_encoding="utf-8"):
        super().__init__(telegram_format)

        #calculate numbers elements in telegram taken from telegram format
        telegram_format_cut = re.sub('[\d<>x=1@]', '', telegram_format)
        
        #set general parameters
        self.string_encoding = string_encoding #default string encoding
        self.len_val = len(telegram_format_cut)
        self.name = telegram_name
        self.len_bytes = super().size
        self.telegram_name = telegram_name
        
        #binding communication_module
        self.comm_module = comm_module #CommModuleSocketTcp class
        if not self.comm_module:
            logging.warning("Telegram %s: no communication module set. NOT USED NOW",telegram_name)
            
        #length check
        if self.len_val == 0:
            raise ValueError("No elements in telegram")
        
        #create nametuple element - data container with parameter access to each element
        try:
            if not telegram_data_names: 
                telegram_data_names = " ".join(list(["v"+str(v) for v in range(self.len_val)]))
            TelegramData = recordclass(telegram_name, telegram_data_names) 
            
        except TypeError:
            raise TypeError("Telegram parameter's string should be 'names' devided by spaces, or empty\(0-n index access\)")
        
        #data check: if no initial data - create: b'' for 's' and 0 for rest of formats
        if not telegram_data:
            telegram_data = [ b'' if k in 'sc' else 0  for k in telegram_format_cut]
            
        self.data = TelegramData._make(telegram_data) 
        
        logging.info("Telegram %s: with format: %s; and variables: %s  - created",telegram_name,telegram_format,telegram_data_names)

    def get_data(self,source_direction) -> "data message": 
        #Doing something to get data
        #in progress
        data = "get data message"
        return data
    def set_string_value(self,value_name,value):
        self.data[value_name] = bytes(value,encoding=self.string_encoding)
        
    def get_string_value(self,value_name,encoding=None):
        if not encoding:
            encoding = self.string_encoding
        return self.data[value_name].decode(encoding)   

    
    def telegram_pack(self):
        binary_data = None
        try:
            #data = map(self.data._asdict().values())
            binary_data = self.pack(*self.data._asdict().values())
        except Exception as ex:
            logging.warning("Telegram %s: _telegram_pack: could not pack telegram: %s",self.telegram_name,ex.args)
        return binary_data
    
    def telegram_unpack(self,binary_data):
        output = None
        try:
            self.data = self.data._make(self.unpack(binary_data))
        except Exception as ex:
            logging.warning("Telegram %s: _telegram_unpack: could not unpack telegram: %s",self.telegram_name,ex.args)
        return output
    

#my_telegram = Telelgram(">ii4s4sff",'Tegegram1001')

#my_telegram.pack(1,2,bytes("AAA!",encoding="utf-8"),bytes("BBB?",encoding="utf-8"),0.11,0.22)

#" ".join(list(["i"+str(v) for v in range(6)]))
#Telegram_1001 = namedtuple('Tegegram1001' , "i4")
#v = [1,2,bytes("AAA!",encoding="utf-8"),bytes("BBB?",encoding="utf-8"),0.11,0.22]

#telegram_1001.val1 = 321
#telegram_1001.val2 = 456
#telegram_1001.sometext1 = bytes("AAA!",encoding="utf-8")
#telegram_1001.sometext2 = bytes("BBB?",encoding="utf-8")
#telegram_1001.float1 = 0.123
#telegram_1001.float2 = 0.456
#telegram_data._asdict()
#
#my_telegram.pack(*telegram_1001._asdict().values())
#
#dict(telegram_data)
#telegram_1001

comm_module = CommModuleSocketTcp("Sock#1") 
my_telegram = Telegram(">ii1s1s",'Telegram1001',"int1 int2 text1 text2")
my_telegram.data.int1 = 2
my_telegram.data.int2 = 3
my_telegram.data.text1 = b"s"
my_telegram.set_string_value("text2","A")

#my_telegram.data.pic = b"\x02\x02\x02\x02\x02\x02\x02\x02\x02\x09"
my_telegram.telegram_pack()

#test_binary_data = b'\x00\x00\x00\x02\x00\x00\x00\x03Q\x09'
#my_telegram.telegram_unpack(test_binary_data)
#my_telegram.data[1]
#my_telegram.get_string_value("text2")

22:24:42: Communication module Sock#1: created
22:24:42: Telegram Telegram1001: no communication module set. NOT USED NOW
22:24:42: Telegram Telegram1001: with format: >ii1s1s; and variables: int1 int2 text1 text2  - created


b'\x00\x00\x00\x02\x00\x00\x00\x03sA'

In [ ]:
partner_plc_telegram_1 = Telegram(">ii1s1s",'Telegram1',"int1 int2 text1 text2")
partner_plc_telegram_1.int1 = 1
partner_plc_telegram_1.int2 = 1
partner_plc_telegram_1.set_string_value("text1","A")
partner_plc_telegram_1.set_string_value("text2","B")
partner_plc_telegram_1.telegram_pack()